# Deploy Open Source Llama 2 in Snowpark Container Services 

### Import Libraries

In [11]:
from snowflake.snowpark.session import Session
from snowflake.ml.model.models import llm
from snowflake.ml.registry import model_registry
from snowflake.ml.model import deploy_platforms
from snowflake.snowpark import VERSION
import snowflake.snowpark.functions as F

import sys
import os
import json
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Set Variables

*Note: Update SNOWFLAKE_WAREHOUSE, SNOWFLAKE_COMPUTE_POOL, HUGGING_FACE_TOKEN before proceeding.*

In [13]:
SNOWFLAKE_DATABASE  = os.getenv("SNOWFLAKE_DATABASE")
SNOWFLAKE_SCHEMA    = os.getenv("SNOWFLAKE_SCHEMA")
SNOWFLAKE_WAREHOUSE = 'DASH_L'
SNOWFLAKE_COMPUTE_POOL = 'DASH_GPU3'
HUGGING_FACE_TOKEN = 'xxxxxxxxxxxxxxxxxx'

### Establish Secure Connection

In [14]:
# Read the login token supplied automatically by Snowflake. These tokens are short lived and should always be read right before creating any new connection.
def get_login_token():
  with open("/snowflake/session/token", "r") as f:
    return f.read()

# Construct Snowflake connection params from environment variables.
def get_connection_params():
  return {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "host": os.getenv("SNOWFLAKE_HOST"),
    "warehouse": SNOWFLAKE_WAREHOUSE,
    "database": SNOWFLAKE_DATABASE,
    "schema": SNOWFLAKE_SCHEMA,
    "authenticator": "oauth",
    "token": get_login_token()
  }

# Create Snowflake Session object
session = Session.builder.configs(get_connection_params()).create()
session.sql_simplifier_enabled = True
snowpark_version = VERSION

# Current Environment Details
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 3.7.0, Python Version: 3.10.11, Platform: Linux-5.4.181-99.354.amzn2.x86_64-x86_64-with-glibc2.35
INFO:snowflake.connector.connection:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
INFO:snowflake.snowpark.session:Snowpark Session information: 
"version" : 1.12.0,
"python.version" : 3.10.11,
"python.connector.version" : 3.7.0,
"python.connector.session.id" : 18047963680641034,
"os.name" : Linux

INFO:snowflake.connector.cursor:query: [alter session set PYTHON_SNOWPARK_USE_SQL_SIMPLIFIER = True]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1


Role                        : "DASH_SPCS"
Database                    : "DASH_DB"
Schema                      : "DASH_SCHEMA"
Warehouse                   : "DASH_L"
Snowpark for Python version : 1.12.0


### Reference Llama 2 from Hugging Face

In [15]:
# Enable INFO log level
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

options = llm.LLMOptions(token=HUGGING_FACE_TOKEN,max_batch_size=100,)
llama_model = llm.LLM(model_id_or_path="meta-llama/Llama-2-7b-chat-hf",options=options)

### Register, Log and Deploy Llama 2 into Snowpark Container Services 

*NOTE: Logging and deploying the same model are one time operations. Once the model is logged and deployed, use ModeReference to get the reference to the model.*

In [16]:
MODEL_NAME    = "LLAMA2_7b_CHAT"
MODEL_VERSION = "BaseV1.1"

registry = model_registry.ModelRegistry(
    session=session, 
    database_name=SNOWFLAKE_DATABASE, 
    schema_name=SNOWFLAKE_SCHEMA, 
    create_if_not_exists=True)

/tmp/ipykernel_97/1196839145.py:4: DeprecationWarning: 
The `snowflake.ml.registry.model_registry.ModelRegistry` has been deprecated starting from version 1.2.0.
It will stay in the Private Preview phase. For future implementations, kindly utilize `snowflake.ml.registry.Registry`,
except when specifically required. The old model registry will be removed once all its primary functionalities are
fully integrated into the new registry.
        
  registry = model_registry.ModelRegistry(
INFO:snowflake.connector.cursor:query: [SHOW DATABASES LIKE 'DASH_DB']
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SHOW SCHEMAS LIKE 'DASH_SCHEMA' IN DATABASE DASH_DB]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SHOW TABLES LIKE '_SYSTEM_REGISTRY_SCHEMA_VERSION' IN DASH_DB.DAS

In [17]:
registry.list_models().to_pandas()

INFO:snowflake.connector.cursor:query: [SELECT * FROM DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_MODELS_VIEW]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 7


,CREATION_CONTEXT,CREATION_ENVIRONMENT_SPEC,CREATION_ROLE,CREATION_TIME,ID,INPUT_SPEC,NAME,OUTPUT_SPEC,RUNTIME_ENVIRONMENT_SPEC,TYPE,URI,VERSION,ARTIFACT_IDS,DESCRIPTION,METRICS,TAGS,REGISTRATION_TIMESTAMP
0,None,"{\n ""python"": ""3.10.13""\n}","""DASH_SPCS""",2023-11-14 15:29:38.425000-08:00,a9b776ce834511ee9c921204c7b8f46d,None,LLAMA2_7b_CHAT,None,None,llm,sfc://DASH_DB.DASH_SCHEMA.SNOWML_MODEL_A9B776CE834511EE9C921204C7B8F46D,BaseV1.0,None,None,None,None,2023-11-14 15:29:39.837000-08:00
1,None,"{\n ""python"": ""3.10.11""\n}","""DASH_SPCS""",2024-03-27 14:02:19.556000-07:00,4a0ec9baec7d11ee95b1369145349ce2,None,LLAMA2_7b_CHAT,None,None,llm,sfc://DASH_DB.DASH_SCHEMA.SNOWML_MODEL_4A0EC9BAEC7D11EE95B1369145349CE2,BaseV1.1,None,None,None,None,2024-03-27 14:02:20.639000-07:00
2,None,"{\n ""python"": ""3.8.16""\n}","""DASH_SPCS""",2023-11-15 11:04:42.250000-08:00,d26762d883e911ee8f5ace7db0e7935f,None,LLAMA2_7b_CHAT,None,None,llm,sfc://DASH_DB.DASH_SCHEMA.SNOWML_MODEL_D26762D883E911EE8F5ACE7DB0E7935F,BestTunedV1.0,None,None,None,None,2023-11-15 11:04:44.853000-08:00
3,None,"{\n ""python"": ""3.9.18""\n}","""DASH_SPCS""",2023-11-14 16:49:14.883000-08:00,c6ca82aa835011eeac5d1204c7b8f46d,None,LLAMA2_7b_CHAT,None,None,llm,sfc://DASH_DB.DASH_SCHEMA.SNOWML_MODEL_C6CA82AA835011EEAC5D1204C7B8F46D,BaseV2.0,None,None,None,None,2023-11-14 16:49:16.186000-08:00
4,None,"{\n ""python"": ""3.9.18""\n}","""DASH_SPCS""",2023-11-27 06:38:13.856000-08:00,934a1d828d3211ee9d301204c7b8f46e,None,LLAMA2_7b_CHAT,None,None,llm,sfc://DASH_DB.DASH_SCHEMA.SNOWML_MODEL_934A1D828D3211EE9D301204C7B8F46E,NewBaseV2.0,None,None,None,None,2023-11-27 06:38:15.860000-08:00
5,None,"{\n ""python"": ""3.10.11""\n}","""DASH_SPCS""",2024-03-06 16:16:55.387000-08:00,ff92534adc1711ee93c05ad1a0b84c26,None,LLAMA2_7b_CHAT,None,None,llm,sfc://DASH_DB.DASH_SCHEMA.SNOWML_MODEL_FF92534ADC1711EE93C05AD1A0B84C26,FineTunedV1.1,None,None,None,None,2024-03-06 16:16:56.386000-08:00
6,None,"{\n ""python"": ""3.9.18""\n}","""DASH_SPCS""",2023-11-16 09:23:06.204000-08:00,c70d0bf284a411eebb681204c7b8f46d,None,LLAMA2_7b_CHAT,None,None,llm,sfc://DASH_DB.DASH_SCHEMA.SNOWML_MODEL_C70D0BF284A411EEBB681204C7B8F46D,NewBaseV1.0,None,None,None,None,2023-11-16 09:23:07.145000-08:00


In [18]:
registry.delete_model(model_name=MODEL_NAME,model_version='BaseV1.1',delete_artifact=True)

INFO:snowflake.connector.cursor:query: [SELECT * FROM DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_MODELS_VIEW]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:query: [SELECT  *  FROM (SELECT * FROM DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_MODELS_VIEW)...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [DELETE FROM DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_MODELS WHERE ID='4a0ec9baec7d11...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [DROP STAGE DASH_DB.DASH_SCHEMA.SNOWML_MODEL_4A0EC9BAEC7D11EE95B1369145349CE2]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT * FROM DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_MODELS_VIEW]
INFO:sno

In [19]:
llama_model_ref = registry.log_model(
    model_name=MODEL_NAME,
    model_version=MODEL_VERSION,
    model=llama_model,
    conda_dependencies=['sentencepiece']
)

INFO:snowflake.connector.cursor:query: [SHOW TABLES LIKE '_SYSTEM_REGISTRY_SCHEMA_VERSION' IN DASH_DB.DASH_SCHEMA]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT MAX(VERSION) AS MAX_VERSION FROM DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_SCH...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT * FROM DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_MODELS_VIEW]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:query: [SELECT count(1) AS "COUNT(LITERAL())" FROM ( SELECT  *  FROM (SELECT * FROM DASH...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [CREATE 

Alternatively, if the model is already deployed, you may reference it using Model Registry

In [20]:
# llama_model_ref = model_registry.ModelReference(
#     registry=registry, 
#     model_name=MODEL_NAME, 
#     model_version=MODEL_VERSION)

*NOTE: Deploying model for the first time can take ~25-30mins*

In [22]:
%%time
llama_model_ref.deploy(
    deployment_name="llama_predict", 
    platform=deploy_platforms.TargetPlatform.SNOWPARK_CONTAINER_SERVICES,
    permanent=True, 
    options={"compute_pool": SNOWFLAKE_COMPUTE_POOL, "num_gpus": 1, "external_access_integrations": ["ALLOW_ALL_ACCESS_INTEGRATION"]})

llama_model_ref = model_registry.ModelReference(registry=registry,model_name=MODEL_NAME,model_version=MODEL_VERSION)
llama_model_ref

INFO:snowflake.connector.cursor:query: [SHOW TABLES LIKE '_SYSTEM_REGISTRY_SCHEMA_VERSION' IN DASH_DB.DASH_SCHEMA]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT MAX(VERSION) AS MAX_VERSION FROM DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_SCH...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [CREATE STAGE IF NOT EXISTS DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_DEPLOYMENTS_STAG...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT * FROM DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_MODELS_VIEW]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:query: [SELECT 

CPU times: user 1.9 s, sys: 356 ms, total: 2.26 s
Wall time: 26min 25s


### Load Data from JSON into Snowflake

*NOTE: Reading data in JSON and storing it in a Snowflake table are one time operations. Once the data is loaded, use Snowpark to load the data from the existing table.*

In [23]:
df = pd.read_json("transcripts_base.json",lines=True)
sf_df = session.write_pandas(df,'transcripts',auto_create_table=True,quote_identifiers=False,overwrite=True)
sf_df.to_pandas().head()

INFO:snowflake.connector.cursor:query: [CREATE TEMP STAGE /* Python:snowflake.connector.pandas_tools.write_pandas() */ y...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ 'file:///tmp/tm...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [CREATE TEMP FILE FORMAT wlpcvmsmre /* Python:snowflake.connector.pandas_tools.wr...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT COLUMN_NAME, TYPE FROM table(infer_schema(location=>'@yydjmskpwy', file_f...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 6
INFO:snowflake.connector.cursor:query: [CREATE  TABLE IF NOT EXISTS vaoolgqsil (LANG

,LANGUAGE,TRANSCRIPT,NAME,LOCATION,TOY_LIST,ID
0,EN,"caller: Hi there, Frosty! I am excited to submit my holiday wish list.\nfrosty: Hello! I'm happy to help you. Can I have your name, please?\ncaller: I am Rachel, and I am calling from Sydney.\nfrosty: Great, Rachel! Now, what's on your wish list?\ncaller: We are thinking of getting the barbie science doll set. But, I'm not quite sure yet.\nfrosty: That's a wonderful choice! May I ask why you're interested in this toy?\ncaller: My daughter loves science and Barbie, so it will be a perfect combo for her.\nfrosty: That sounds perfect indeed. How do you and your family plan to celebrate the holiday season?\ncaller: We are planning a small gathering with close family and friends and lots of food and games.\nfrosty: That sounds like a lovely time. Before we confirm your wish list, would you like to explore some other options for your daughter?\ncaller: Maybe one more option?\nfrosty: How about the Beast Lab: Shark Beast Creator? It's a fantastic creative and educational toy that your daughter might enjoy, especially if she loves science.\ncaller: Oh, that sounds interesting. But I think we'll stick with the barbie science doll set.\nfrosty: Alright, so the Barbie Science Lab Playset will be on the wish list. Thank you for sharing your holiday plans, and I hope your family has a fantastic time. Have a great day, Rachel!\ncaller: Thank you, Frosty! Happy holidays!",Rachel,Sydney,"[\n ""Barbie Science Lab Playset""\n]",0
1,EN,"caller: Hi Frosty! My name is Jessica, and I live in Los Angeles. I need some help finding the perfect toy for my little one.\nfrosty: Hello, Jessica! It's great to have you here. Let's find that perfect toy! What are some of your child's interests or hobbies?\ncaller: She loves playing dress-up and pretend play. She's also quite artistic.\nfrosty: That's delightful! I have a couple of suggestions: the Barbie Dreamhouse 2023 or the Calico Critters Pony's Stylish Hair Salon. Both encourage imaginative play and creativity.\ncaller: Hmm, I like both options. Maybe the new Barbie Dreamhouse?\nfrosty: The Barbie Dreamhouse 2023 is an excellent choice for hours of endless imaginative play. Before we confirm, is there any other toy you'd like to consider?\ncaller: No, I think we are good with the Barbie Dreamhouse.\nfrosty: Fantastic! The Barbie Dreamhouse 2023 is now on your holiday wish list. What's your favorite holiday memory, Jessica?\ncaller: My favorite memory is when we all went to see the city's holiday lights display. It was magical!\nfrosty: That sounds enchanting! I hope you and your family create more beautiful memories this holiday season. Have a great day, Jessica!\ncaller: Thank you, Frosty! Happy holidays!",Jessica,Los Angeles,"[\n ""Barbie Dreamhouse 2023""\n]",2
2,EN,"caller: Hi, Frosty! My name is Ashley, and I'm calling from Auckland. I need to pick a gift for my nephew, but I'm a little unsure.\nfrosty: Hello, Ashley! I'm here to help you find the perfect gift. What are your nephew's interests or hobbies?\ncaller: He is really into vehicles and action figures.\nfrosty: Great! I have a couple of options for you: The Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Van or the Bluey Convertible and Figures. Which one sounds more appealing to you?\ncaller: The ninja turtles delivery van seems perfect! Let's add that to the list.\nfrosty: Excellent choice! The Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Van is now on your wish list. Can you tell me about one of your favorite pastimes or hobbies, Ashley?\ncaller: Sure! I love spending time in nature, going for hikes and exploring new trails.\nfrosty: That sounds like an excellent way to unwind and enjoy the world around you. I hope you have a fantastic holiday season, Ashley! Goodbye!\ncaller: Thank you, Frosty! Happy holidays!",Ashley,Auckland,"[\n ""Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Van""\n]",4
3,EN,"caller: Hey Frosty! I need some help

### Simple Prompt Engineering Example

For every transcript, define summarization instruction for the LLM

In [24]:
begin_prompt = \
"""
[INST] Summarize this transcript in less than 200 words: 
"""
end_prompt = " [/INST]"

df_inputs = sf_df.with_column('"input"',F.concat_ws(F.lit(" "),F.lit(begin_prompt),F.col('transcript'),F.lit(end_prompt))).select('"input"')
df_inputs.to_pandas().head()

INFO:snowflake.connector.cursor:query: [SELECT  *  FROM transcripts]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:query: [SELECT concat_ws(' ', '\n[INST] Summarize this transcript in less than 200 words...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 69


,input
0,"\n[INST] Summarize this transcript in less than 200 words: \n caller: Hi there, Frosty! I am excited to submit my holiday wish list.\nfrosty: Hello! I'm happy to help you. Can I have your name, please?\ncaller: I am Rachel, and I am calling from Sydney.\nfrosty: Great, Rachel! Now, what's on your wish list?\ncaller: We are thinking of getting the barbie science doll set. But, I'm not quite sure yet.\nfrosty: That's a wonderful choice! May I ask why you're interested in this toy?\ncaller: My daughter loves science and Barbie, so it will be a perfect combo for her.\nfrosty: That sounds perfect indeed. How do you and your family plan to celebrate the holiday season?\ncaller: We are planning a small gathering with close family and friends and lots of food and games.\nfrosty: That sounds like a lovely time. Before we confirm your wish list, would you like to explore some other options for your daughter?\ncaller: Maybe one more option?\nfrosty: How about the Beast Lab: Shark Beast Creator? It's a fantastic creative and educational toy that your daughter might enjoy, especially if she loves science.\ncaller: Oh, that sounds interesting. But I think we'll stick with the barbie science doll set.\nfrosty: Alright, so the Barbie Science Lab Playset will be on the wish list. Thank you for sharing your holiday plans, and I hope your family has a fantastic time. Have a great day, Rachel!\ncaller: Thank you, Frosty! Happy holidays! [/INST]"
1,"\n[INST] Summarize this transcript in less than 200 words: \n caller: Hi Frosty! My name is Jessica, and I live in Los Angeles. I need some help finding the perfect toy for my little one.\nfrosty: Hello, Jessica! It's great to have you here. Let's find that perfect toy! What are some of your child's interests or hobbies?\ncaller: She loves playing dress-up and pretend play. She's also quite artistic.\nfrosty: That's delightful! I have a couple of suggestions: the Barbie Dreamhouse 2023 or the Calico Critters Pony's Stylish Hair Salon. Both encourage imaginative play and creativity.\ncaller: Hmm, I like both options. Maybe the new Barbie Dreamhouse?\nfrosty: The Barbie Dreamhouse 2023 is an excellent choice for hours of endless imaginative play. Before we confirm, is there any other toy you'd like to consider?\ncaller: No, I think we are good with the Barbie Dreamhouse.\nfrosty: Fantastic! The Barbie Dreamhouse 2023 is now on your holiday wish list. What's your favorite holiday memory, Jessica?\ncaller: My favorite memory is when we all went to see the city's holiday lights display. It was magical!\nfrosty: That sounds enchanting! I hope you and your family create more beautiful memories this holiday season. Have a great day, Jessica!\ncaller: Thank you, Frosty! Happy holidays! [/INST]"
2,"\n[INST] Summarize this transcript in less than 200 words: \n caller: Hi, Frosty! My name is Ashley, and I'm calling from Auckland. I need to pick a gift for my nephew, but I'm a little unsure.\nfrosty: Hello, Ashley! I'm here to help you find the perfect gift. What are your nephew's interests or hobbies?\ncaller: He is really into vehicles and action figures.\nfrosty: Great! I have a couple of options for you: The Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Van or the Bluey Convertible and Figures. Which one sounds more appealing to you?\ncaller: The ninja turtles delivery van seems perfect! Let's add that to the list.\nfrosty: Excellent choice! The Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Van is now on your wish list. Can you tell me about one of your favorite pastimes or hobbies, Ashley?\ncaller: Sure! I love spending time in nature, going for hikes and exploring new trails.\nfrosty: That sounds like an excellent way to unwind and enjoy the world around you. I hope you have a fantastic holiday season, Ashley! Goodbye!\ncaller: Thank you, Frosty! Happy holidays! [/INST]"
3,"\n[INST] Summarize this transcript in less than 200 words: \n caller: Hey Frosty! I need so

### Inference using Simple Prompt

Pass the summariation instruction to the LLM and examine results of 10 records

In [26]:
df_predict_results = llama_model_ref.predict(deployment_name="llama_predict",data=df_inputs)
df_predict_results.select('"input"','"generated_text"').limit(5).to_pandas()

INFO:snowflake.connector.cursor:query: [SHOW TABLES LIKE '_SYSTEM_REGISTRY_SCHEMA_VERSION' IN DASH_DB.DASH_SCHEMA]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT MAX(VERSION) AS MAX_VERSION FROM DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_SCH...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT * FROM DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_DEPLOYMENTS_VIEW]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:query: [SELECT count(1) AS "COUNT(LITERAL())" FROM ( SELECT "MODEL_NAME", "MODEL_VERSION...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SE

,input,generated_text
0,"\n[INST] Summarize this transcript in less than 200 words: \n caller: Hi there, Frosty! I am excited to submit my holiday wish list.\nfrosty: Hello! I'm happy to help you. Can I have your name, please?\ncaller: I am Rachel, and I am calling from Sydney.\nfrosty: Great, Rachel! Now, what's on your wish list?\ncaller: We are thinking of getting the barbie science doll set. But, I'm not quite sure yet.\nfrosty: That's a wonderful choice! May I ask why you're interested in this toy?\ncaller: My daughter loves science and Barbie, so it will be a perfect combo for her.\nfrosty: That sounds perfect indeed. How do you and your family plan to celebrate the holiday season?\ncaller: We are planning a small gathering with close family and friends and lots of food and games.\nfrosty: That sounds like a lovely time. Before we confirm your wish list, would you like to explore some other options for your daughter?\ncaller: Maybe one more option?\nfrosty: How about the Beast Lab: Shark Beast Creator? It's a fantastic creative and educational toy that your daughter might enjoy, especially if she loves science.\ncaller: Oh, that sounds interesting. But I think we'll stick with the barbie science doll set.\nfrosty: Alright, so the Barbie Science Lab Playset will be on the wish list. Thank you for sharing your holiday plans, and I hope your family has a fantastic time. Have a great day, Rachel!\ncaller: Thank you, Frosty! Happy holidays! [/INST]","Rachel from Sydney calls to submit her holiday wish list to Frosty. She is considering the Barbie Science Lab Playset for her daughter, who enjoys science and Barbie. Frosty suggests an alternative, the Beast Lab: Shark Beast Creator, which is a creative and educational toy. Rachel decides to stick with the original choice and thanks Frosty for his help."
1,"\n[INST] Summarize this transcript in less than 200 words: \n caller: Hi Frosty! My name is Jessica, and I live in Los Angeles. I need some help finding the perfect toy for my little one.\nfrosty: Hello, Jessica! It's great to have you here. Let's find that perfect toy! What are some of your child's interests or hobbies?\ncaller: She loves playing dress-up and pretend play. She's also quite artistic.\nfrosty: That's delightful! I have a couple of suggestions: the Barbie Dreamhouse 2023 or the Calico Critters Pony's Stylish Hair Salon. Both encourage imaginative play and creativity.\ncaller: Hmm, I like both options. Maybe the new Barbie Dreamhouse?\nfrosty: The Barbie Dreamhouse 2023 is an excellent choice for hours of endless imaginative play. Before we confirm, is there any other toy you'd like to consider?\ncaller: No, I think we are good with the Barbie Dreamhouse.\nfrosty: Fantastic! The Barbie Dreamhouse 2023 is now on your holiday wish list. What's your favorite holiday memory, Jessica?\ncaller: My favorite memory is when we all went to see the city's holiday lights display. It was magical!\nfrosty: That sounds enchanting! I hope you and your family create more beautiful memories this holiday season. Have a great day, Jessica!\ncaller: Thank you, Frosty! Happy holidays! [/INST]","Jessica, a caller from Los Angeles, is looking for a toy for her child that promotes imaginative play and creativity. Frosty suggests the Barbie Dreamhouse 2023 or the Calico Critters Pony's Stylish Hair Salon. Jessica decides on the Barbie Dreamhouse, and Frosty asks about her favorite holiday memories. Jessica shares a memory of seeing the city's holiday lights"
2,"\n[INST] Summarize this transcript in less than 200 words: \n caller: Hi, Frosty! My name is Ashley, and I'm calling from Auckland. I need to pick a gift for my nephew, but I'm a little unsure.\nfrosty: Hello, Ashley! I'm here to help you find the perfect gift. What are your nephew's interests or hobbies?\ncaller: He is really into vehicles and action figures.\nfrosty: Great! I have a couple of options for you: The Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Va

## Cleanup Resources

Delete base model service

In [27]:
session.sql(f"show services in compute pool {SNOWFLAKE_COMPUTE_POOL}").show();

INFO:snowflake.connector.cursor:query: [show services in compute pool DASH_GPU3]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 2


------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"name"                                    |"database_name"  |"schema_name"  |"owner"    |"compute_pool"  |"dns_name"                                          |"min_instances"  |"max_instances"  |"auto_resume"  |"external_access_integrations"    |"created_on"                      |"updated_on"                      |"resumed_on"  |"comment"  |"owner_role_type"  |"query_warehouse"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [29]:
# Replace SERVICE_XXXXXXXXXXXXXXXXXXXXXX with the one from running the above cell
service_id = 'SERVICE_D997D326EC8511EE95B1369145349CE2'
session.sql(f"drop service {service_id}").collect();

INFO:snowflake.connector.cursor:query: [drop service SERVICE_D997D326EC8511EE95B1369145349CE2]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1


In [30]:
session.sql(f"show services in compute pool {SNOWFLAKE_COMPUTE_POOL}").show();

INFO:snowflake.connector.cursor:query: [show services in compute pool DASH_GPU3]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1


-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"name"       |"database_name"  |"schema_name"  |"owner"    |"compute_pool"  |"dns_name"                                          |"min_instances"  |"max_instances"  |"auto_resume"  |"external_access_integrations"    |"created_on"                      |"updated_on"                      |"resumed_on"  |"comment"  |"owner_role_type"  |"query_warehouse"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Fine-tune Llama 2 Model

Fine-tuning is one form of model training. We start training from a pre-trained model and adjust a set of model parameters to better solve for a concrete task based on task specific data. Today we are going to fine-tune 7B Llama 2 model using LoRA (Low-Rank Adaptation)--which is a parameter efficient way of fine-tuning LLM.

### Import Libraries

In [31]:
from datasets import Dataset, ClassLabel
from transformers import AutoTokenizer, AutoModelForCausalLM
import sys
from utils import Concatenator
import pandas as pd
pd.set_option('display.max_colwidth', None)

import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
import sentencepiece
import os
import json
from transformers import TrainerCallback
from contextlib import nullcontext
from transformers import default_data_collator, Trainer, TrainingArguments

from snowflake.snowpark.session import Session
from snowflake.snowpark import VERSION
import snowflake.snowpark.functions as F
from snowflake.ml.registry import model_registry
from snowflake.ml.model import deploy_platforms
from snowflake.ml.model.models import llm

import logging 
logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)
logger = logging.getLogger("snowflake.ml")
logger.setLevel(logging.ERROR)

### Set Hugging Face Token

In [32]:
!huggingface-cli login --token $HUGGING_FACE_TOKEN

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Load Pre-trained Llama 2 Model From Hugging Face

In [34]:
model_id="meta-llama/Llama-2-7b-chat-hf"
print('loading tokenizer')
tokenizer = LlamaTokenizer.from_pretrained(model_id)
print('loading model')
model = LlamaForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map='auto', torch_dtype=torch.float16)

loading tokenizer
loading model


INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Load and Split Dataset

In [21]:
# def prepare_stratified_dataset(path, seed = 42):
#     raw_df = pd.read_json(path, lines=True)
#     raw_df['id'] = raw_df.index
#     ds = Dataset.from_pandas(raw_df, split='train')
#     cl = ClassLabel(num_classes=4, names=["EN", "FR", "DE", "ES"])
#     new_features = ds.features.copy()
#     new_features['lang_label'] = cl
#     cl_d = {l : cl.str2int(l) for l in ["EN", "FR", "DE", "ES"]}
#     def convert_lang(sample):
#         sample['lang_label'] = cl_d[sample['language']]
#         return sample
#     ds = ds.map(convert_lang, features=new_features)
#     ds_split = ds.train_test_split(test_size=0.15, stratify_by_column='lang_label', seed=42)
#     test_ds_split = ds_split['test'].train_test_split(test_size=2/3, stratify_by_column='lang_label', seed=42)
#     return ds_split['train'].to_pandas(), test_ds_split['train'].to_pandas(), test_ds_split['test'].to_pandas()

# train_df, eval_df, test_df = prepare_stratified_dataset(
#     'transcripts.json'
# )

In [35]:
df = pd.read_json("transcripts_inst.json", lines=True)
print(f"Total records: {df.shape[0]}")
train_df = df.head(100)
print(f"Train        : {train_df.shape[0]}")
eval_df = df[200:300]
print(f"Eval         : {eval_df.shape[0]}")
test_df = df.tail(100)
print(f"Test         : {test_df.shape[0]}")

Total records: 1368
Train        : 100
Eval         : 100
Test         : 100


In [36]:
train_df.head()

,instruction,input,output,language
0,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""caller: Hi there, Frosty! I am excited to submit my holiday wish list.\nfrosty: Hello! I'm happy to help you. Can I have your name, please?\ncaller: I am Rachel, and I am calling from Sydney.\nfrosty: Great, Rachel! Now, what's on your wish list?\ncaller: We are thinking of getting the barbie science doll set. But, I'm not quite sure yet.\nfrosty: That's a wonderful choice! May I ask why you're interested in this toy?\ncaller: My daughter loves science and Barbie, so it will be a perfect combo for her.\nfrosty: That sounds perfect indeed. How do you and your family plan to celebrate the holiday season?\ncaller: We are planning a small gathering with close family and friends and lots of food and games.\nfrosty: That sounds like a lovely time. Before we confirm your wish list, would you like to explore some other options for your daughter?\ncaller: Maybe one more option?\nfrosty: How about the Beast Lab: Shark Beast Creator? It's a fantastic creative and educational toy that your daughter might enjoy, especially if she loves science.\ncaller: Oh, that sounds interesting. But I think we'll stick with the barbie science doll set.\nfrosty: Alright, so the Barbie Science Lab Playset will be on the wish list. Thank you for sharing your holiday plans, and I hope your family has a fantastic time. Have a great day, Rachel!\ncaller: Thank you, Frosty! Happy holidays!""","{""toy_list"": [""Barbie Science Lab Playset""], ""location"": ""Sydney""}",EN
1,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""caller: Hi Frosty! My name is Jessica, and I live in Los Angeles. I need some help finding the perfect toy for my little one.\nfrosty: Hello, Jessica! It's great to have you here. Let's find that perfect toy! What are some of your child's interests or hobbies?\ncaller: She loves playing dress-up and pretend play. She's also quite artistic.\nfrosty: That's delightful! I have a couple of suggestions: the Barbie Dreamhouse 2023 or the Calico Critters Pony's Stylish Hair Salon. Both encourage imaginative play and creativity.\ncaller: Hmm, I like both options. Maybe the new Barbie Dreamhouse?\nfrosty: The Barbie Dreamhouse 2023 is an excellent choice for hours of endless imaginative play. Before we confirm, is there any other toy you'd like to consider?\ncaller: No, I think we are good with the Barbie Dreamhouse.\nfrosty: Fantastic! The Barbie Dreamhouse 2023 is now on your holiday wish list. What's your favorite holiday memory, Jessica?\ncaller: My favorite memory is when we all went to see the city's holiday lights display. It was magical!\nfrosty: That sounds enchanting! I hope you and your family create more beautiful memories this holiday season. Have a great day, Jessica!\ncaller: Thank you, Frosty! Happy holidays!""","{""toy_list"": [""Barbie Dreamhouse 2023""], ""location"": ""Los Angeles""}",EN
2,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""caller: Hi, Frosty! My name is Ashley, and I'm calling from Auckland. I need to pick a gift for my nephew, but I'm a little unsure.\nfrosty: Hello, Ashley! I'm here to help you find the perfect gift. What are your nephew's interests or hobbies?\ncaller: He is really into vehicles and action figures.\nfrosty: Great! I have a couple of options for you: The Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Delivery Van or the Bluey Convertible and Figures. Which one sounds more appealing to you?\ncaller: The ninja turtles delivery van seems perfect! Let's add that to the list.\nfrosty: Excellent choice! The Teenage Mutant Ninja Turtles: Mutant Mayhem Pizza Fire Deliver

In [37]:
datasets = {
    'train': Dataset.from_pandas(train_df),
    'eval': Dataset.from_pandas(eval_df),
    'test': Dataset.from_pandas(test_df)
}

### Apply Prompt to Datasets

In [38]:
train_prompt = f"""
[INST] <<SYS>>
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
<</SYS>>
### Instruction:
{{instruction}}
### Input:
{{input_}}
### Output:
{{output}}
{{eos_token}}
"""

eval_prompt = f"""
[INST] <<SYS>>
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
<</SYS>>
### Instruction:
{{instruction}}
### Input:
{{input_}}
### Output:
"""

In [39]:
def apply_train_template(sample):
    return {
        "text": train_prompt.format(
            instruction=sample["instruction"],
            input_=sample["input"].replace('\\n', '\n'),
            output=sample["output"],
            eos_token=tokenizer.eos_token,
        )
    }

def apply_eval_template(sample):
    return {
        "text": eval_prompt.format(
            instruction=sample["instruction"],
            input_=sample["input"].replace('\\n', '\n')
        )
    }

#applying template

datasets['train'] = datasets['train'].map(apply_train_template, remove_columns=list(datasets['train'].features))
for k in ['eval', 'test']:
    datasets[k] = datasets[k].map(apply_eval_template, remove_columns=list(datasets[k].features))

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

### Tokenize Datasets

In [40]:
for k, v in datasets.items():
    datasets[k] = v.map(
        lambda sample: tokenizer(sample["text"]),
        batched=True,
        
        remove_columns=list(v.features),
    ).map(Concatenator(), batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

### Fine-tune Configuration

Instead of adjusting all the ~7B parameters, LoRA allows us to adjust only a percent of model weights--which can save compute and memory resources dramatically. For this lab, we will fine-tune our model using LoRA on a single A10 GPU. This will demostrate how good the inference can be on fine-tuned models even with limited compute.

In [41]:
#setting the model into training mode
model.train()

#setting up training
def create_peft_config(model):
    from peft import (
        get_peft_model,
        LoraConfig,
        TaskType,
        prepare_model_for_kbit_training,
    )

    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,
        lora_alpha=32,
        lora_dropout=0.05,
        target_modules = ["q_proj", "v_proj"]
    )

    # prepare int-8 model for training
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model, peft_config

# create peft config
model, lora_config = create_peft_config(model)

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


In [42]:
# !rm -r output_weights_dir # deletes prior fine tuning weights
!mkdir output_weights_dir

We're passing train_dataset and eval_dataset that are used to generate loss calculation during fine-tuning process and we've set output_weights_dir as the directory where the fine-tuned weights will be stored after fine-tuning job completes.

To achieve good performance for the task, you will need at least 1 num_epochs.

In [43]:
output_dir = "output_weights_dir"
enable_profiler = False

config = {
    'lora_config': lora_config,
    'learning_rate': 1e-4,
    'num_train_epochs': 1,
    'gradient_accumulation_steps': 2,
    'per_device_train_batch_size': 2,
    'gradient_checkpointing': False,
}

# Define training args
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    bf16=True,  # Use BF16 if available
    # logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
    optim="adamw_torch_fused",
    max_steps=total_steps if enable_profiler else -1,
    **{k:v for k,v in config.items() if k != 'lora_config'}
)

In [44]:
profiler = nullcontext() 

with profiler:
    # Create Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=datasets['train'],
        eval_dataset=datasets['eval'],
        data_collator=default_data_collator,
        callbacks=[profiler_callback] if enable_profiler else [],
    )

    # Start training
    trainer.train()
    
model.save_pretrained(output_dir)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/envs/rapids/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/envs/rapids/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss


### Log and Deploy Fine-tuned Llama 2

In [45]:
MODEL_NAME = "LLAMA2_7b_CHAT"
MODEL_VERSION = "FineTunedV1.1"
DEPLOYMENT_NAME = "FINETUNED_LLAMA2"

registry = model_registry.ModelRegistry(
    session=session, 
    database_name=SNOWFLAKE_DATABASE, 
    schema_name=SNOWFLAKE_SCHEMA, 
    create_if_not_exists=True)

/tmp/ipykernel_97/41704961.py:5: DeprecationWarning: 
The `snowflake.ml.registry.model_registry.ModelRegistry` has been deprecated starting from version 1.2.0.
It will stay in the Private Preview phase. For future implementations, kindly utilize `snowflake.ml.registry.Registry`,
except when specifically required. The old model registry will be removed once all its primary functionalities are
fully integrated into the new registry.
        
  registry = model_registry.ModelRegistry(
INFO:snowflake.connector.cursor:query: [SHOW DATABASES LIKE 'DASH_DB']
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SHOW SCHEMAS LIKE 'DASH_SCHEMA' IN DATABASE DASH_DB]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SHOW TABLES LIKE '_SYSTEM_REGISTRY_SCHEMA_VERSION' IN DASH_DB.DASH_

In [46]:
registry.list_deployments(model_name=MODEL_NAME,model_version=MODEL_VERSION).to_pandas()

INFO:snowflake.connector.cursor:query: [SELECT * FROM DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_DEPLOYMENTS_VIEW]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:query: [SELECT "MODEL_NAME", "MODEL_VERSION", "DEPLOYMENT_NAME", "CREATION_TIME", "TARGE...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 0


,MODEL_NAME,MODEL_VERSION,DEPLOYMENT_NAME,CREATION_TIME,TARGET_METHOD,TARGET_PLATFORM,SIGNATURE,OPTIONS,STAGE_PATH,ROLE


In [47]:
# registry.delete_deployment(model_name=MODEL_NAME,model_version=MODEL_VERSION,deployment_name=DEPLOYMENT_NAME)

In [48]:
registry.list_models().to_pandas()

INFO:snowflake.connector.cursor:query: [SELECT * FROM DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_MODELS_VIEW]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 7


,CREATION_CONTEXT,CREATION_ENVIRONMENT_SPEC,CREATION_ROLE,CREATION_TIME,ID,INPUT_SPEC,NAME,OUTPUT_SPEC,RUNTIME_ENVIRONMENT_SPEC,TYPE,URI,VERSION,ARTIFACT_IDS,DESCRIPTION,METRICS,TAGS,REGISTRATION_TIMESTAMP
0,None,"{\n ""python"": ""3.10.13""\n}","""DASH_SPCS""",2023-11-14 15:29:38.425000-08:00,a9b776ce834511ee9c921204c7b8f46d,None,LLAMA2_7b_CHAT,None,None,llm,sfc://DASH_DB.DASH_SCHEMA.SNOWML_MODEL_A9B776CE834511EE9C921204C7B8F46D,BaseV1.0,None,None,None,None,2023-11-14 15:29:39.837000-08:00
1,None,"{\n ""python"": ""3.8.16""\n}","""DASH_SPCS""",2023-11-15 11:04:42.250000-08:00,d26762d883e911ee8f5ace7db0e7935f,None,LLAMA2_7b_CHAT,None,None,llm,sfc://DASH_DB.DASH_SCHEMA.SNOWML_MODEL_D26762D883E911EE8F5ACE7DB0E7935F,BestTunedV1.0,None,None,None,None,2023-11-15 11:04:44.853000-08:00
2,None,"{\n ""python"": ""3.9.18""\n}","""DASH_SPCS""",2023-11-14 16:49:14.883000-08:00,c6ca82aa835011eeac5d1204c7b8f46d,None,LLAMA2_7b_CHAT,None,None,llm,sfc://DASH_DB.DASH_SCHEMA.SNOWML_MODEL_C6CA82AA835011EEAC5D1204C7B8F46D,BaseV2.0,None,None,None,None,2023-11-14 16:49:16.186000-08:00
3,None,"{\n ""python"": ""3.9.18""\n}","""DASH_SPCS""",2023-11-27 06:38:13.856000-08:00,934a1d828d3211ee9d301204c7b8f46e,None,LLAMA2_7b_CHAT,None,None,llm,sfc://DASH_DB.DASH_SCHEMA.SNOWML_MODEL_934A1D828D3211EE9D301204C7B8F46E,NewBaseV2.0,None,None,None,None,2023-11-27 06:38:15.860000-08:00
4,None,"{\n ""python"": ""3.10.11""\n}","""DASH_SPCS""",2024-03-06 16:16:55.387000-08:00,ff92534adc1711ee93c05ad1a0b84c26,None,LLAMA2_7b_CHAT,None,None,llm,sfc://DASH_DB.DASH_SCHEMA.SNOWML_MODEL_FF92534ADC1711EE93C05AD1A0B84C26,FineTunedV1.1,None,None,None,None,2024-03-06 16:16:56.386000-08:00
5,None,"{\n ""python"": ""3.10.11""\n}","""DASH_SPCS""",2024-03-27 15:03:33.324000-07:00,d997d326ec8511ee95b1369145349ce2,None,LLAMA2_7b_CHAT,None,None,llm,sfc://DASH_DB.DASH_SCHEMA.SNOWML_MODEL_D997D326EC8511EE95B1369145349CE2,BaseV1.1,None,None,None,None,2024-03-27 15:03:34.479000-07:00
6,None,"{\n ""python"": ""3.9.18""\n}","""DASH_SPCS""",2023-11-16 09:23:06.204000-08:00,c70d0bf284a411eebb681204c7b8f46d,None,LLAMA2_7b_CHAT,None,None,llm,sfc://DASH_DB.DASH_SCHEMA.SNOWML_MODEL_C70D0BF284A411EEBB681204C7B8F46D,NewBaseV1.0,None,None,None,None,2023-11-16 09:23:07.145000-08:00


In [56]:
registry.delete_model(model_name=MODEL_NAME,model_version='FineTunedV1.1',delete_artifact=True)

INFO:snowflake.connector.cursor:query: [SELECT * FROM DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_MODELS_VIEW]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:query: [SELECT  *  FROM (SELECT * FROM DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_MODELS_VIEW)...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [DELETE FROM DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_MODELS WHERE ID='25071a9eec8e11...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [DROP STAGE DASH_DB.DASH_SCHEMA.SNOWML_MODEL_25071A9EEC8E11EE95B1369145349CE2]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT * FROM DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_MODELS_VIEW]
INFO:sno

#### Reference Llama 2 from Fine-tuning

In [57]:
# Referencing our fine-tuned weights and using the Hugging Face token to merge them with base Llama 2 model

options = llm.LLMOptions(
    token=HUGGING_FACE_TOKEN,
    max_batch_size=100,
)

llama_model = llm.LLM(
    model_id_or_path='output_weights_dir',
    options=options
)

#### Log and Deploy fine-tuned Llama 2 model using Model Registry

Logging and deploying fine-tuned model with our setup may take ~15mins.

In [58]:
%%time
# log model in registry
llama_model_ref = registry.log_model(
    model_name=MODEL_NAME,
    model_version=MODEL_VERSION,
    model=llama_model,
    conda_dependencies=['sentencepiece']
)

# deploy model
llama_model_ref.deploy(
    deployment_name=DEPLOYMENT_NAME, 
    platform=deploy_platforms.TargetPlatform.SNOWPARK_CONTAINER_SERVICES,
    permanent=True, 
    options={"compute_pool": SNOWFLAKE_COMPUTE_POOL, "num_gpus": 1, "external_access_integrations": ["ALLOW_ALL_ACCESS_INTEGRATION"]})

INFO:snowflake.connector.cursor:query: [SHOW TABLES LIKE '_SYSTEM_REGISTRY_SCHEMA_VERSION' IN DASH_DB.DASH_SCHEMA]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT MAX(VERSION) AS MAX_VERSION FROM DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_SCH...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT * FROM DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_MODELS_VIEW]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:query: [SELECT count(1) AS "COUNT(LITERAL())" FROM ( SELECT  *  FROM (SELECT * FROM DASH...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [CREATE 

CPU times: user 1.97 s, sys: 0 ns, total: 1.97 s
Wall time: 10min 45s


{'name': 'DASH_DB.DASH_SCHEMA.FINETUNED_LLAMA2',
 'platform': <TargetPlatform.SNOWPARK_CONTAINER_SERVICES: 'SNOWPARK_CONTAINER_SERVICES'>,
 'target_method': 'infer',
 'signature': ModelSignature(
                     inputs=[
                         FeatureSpec(dtype=DataType.STRING, name='input')
                     ],
                     outputs=[
                         FeatureSpec(dtype=DataType.STRING, name='generated_text')
                     ]
                 ),
 'options': {'compute_pool': 'DASH_GPU3',
  'num_gpus': 1,
  'external_access_integrations': ['ALLOW_ALL_ACCESS_INTEGRATION']},
 'details': {'service_info': {'name': 'SERVICE_8BFE1284EC8E11EE95B1369145349CE2',
   'database_name': 'DASH_DB',
   'schema_name': 'DASH_SCHEMA',
   'owner': 'DASH_SPCS',
   'compute_pool': 'DASH_GPU3',
   'spec': '---\nspec:\n  containers:\n  - name: "inference-server"\n    image: "sfsenorthamerica-build-spcs.registry.snowflakecomputing.com/dash_db/dash_schema/snowml_repo/4d71df997ca151d

Alternatively, if the model is already deployed, you may reference it using Model Registry

In [59]:
# llama_model_ref = model_registry.ModelReference(
#     registry=registry, 
#     model_name=MODEL_NAME, 
#     model_version=MODEL_VERSION)

### Inference on Eval Dataset using fine-tuned Llama 2

In [60]:
eval_df['input'] = eval_df.apply(
    lambda x: eval_prompt.format(
        instruction=x["instruction"],
        input_=x["input"].replace('\\n', '\n')
    ), axis=1
)
eval_df.reset_index(drop=True, inplace=True)
eval_df.head()

/tmp/ipykernel_97/3143168716.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_df['input'] = eval_df.apply(


,instruction,input,output,language
0,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"\n[INST] <<SYS>>\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n<</SYS>>\n### Instruction:\nExtract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.\n### Input:\n""caller: Hey, I'm Jack, and I'm looking for a toy for my niece.\nfrosty: Hello Jack! I'm Frosty, and I'd be happy to help you find the perfect toy. Where are you calling from?\ncaller: I'm calling from Cape Town, South Africa.\nfrosty: Thank you, Jack from Cape Town! Can you tell me a bit about your niece's interests so we can find a great toy for her?\ncaller: She's really into animals and likes playing pretend with her toys.\nfrosty: How about the Fisher-Price Little People Mickey and Friends Playset? This playset features animal characters and is perfect for encouraging imaginative play.\ncaller: That sounds nice, but I think she'd prefer something focused on real animals.\nfrosty: In that case, the Calico Critters Pony's Stylish Hair Salon may be just the toy you're looking for. It comes with pony figurines and a cute hair salon for them.\ncaller: Oh, that sounds perfect! Let's go with the pony's hair salon.\nfrosty: Great choice! We've added the Calico Critters Pony's Stylish Hair Salon to your holiday wish list. Before we finish though, can you tell me how you plan to celebrate the holiday season?\ncaller: Sure, I'll be spending the holidays at the beach with my family and enjoying the warm weather.\nfrosty: That sounds amazing! To confirm, your toy list includes the Calico Critters Pony's Stylish Hair Salon. Is that correct?\ncaller: Yes, that's correct. Thank you!+\nfrosty: You're welcome, Jack! It's been a pleasure assisting you. Have a fantastic holiday season!""\n### Output:\n","{""toy_list"": [""Calico Critters Pony's Stylish Hair Salon""], ""location"": ""Cape Town""}",EN
1,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"\n[INST] <<SYS>>\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n<</SYS>>\n### Instruction:\nExtract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.\n### Input:\n""caller: Hi Frosty, I'm Emily.\nfrosty: Hello Emily! I'm here to help you with your holiday wish list. Where are you calling from?\ncaller: I'm calling from Sydney, Australia.\nfrosty: Thank you, Emily from Sydney! What's the first item you'd like to add to your wish list?\ncaller: I'm looking for a toy that's educational and fun for my son. He's really into science and technology.\nfrosty: How about the Thames & Kosmos Candy Vending Machine STEM experiment kit? It combines science and technology with creating a candy vending machine!\ncaller: That sounds great, but I'd like to explore some more options before deciding.\nfrosty: Sure thing! We also have the LeapFrog Magic Adventures Microscope, which allows kids to explore the world of science up-close with a real working microscope.\ncaller: I think the magic microscope is the one! Let's add that to my wish list.\nfrosty: Fantastic choice! I've added the LeapFrog Magic Adventures Microscope to your list. Before we wrap up, tell me a bit about your favorite way to spend time during the holiday season.\ncaller: I enjoy baking holiday treats and watching seasonal movies with my family.\nfrosty: That sounds delightful! To confirm, your toy list includes the LeapFrog Magic Adventures Microsc

In [61]:
eval_df['predicted'] = llama_model_ref.predict(deployment_name=DEPLOYMENT_NAME,data=eval_df)
eval_df[['output', 'predicted']].head()

INFO:snowflake.connector.cursor:query: [SHOW TABLES LIKE '_SYSTEM_REGISTRY_SCHEMA_VERSION' IN DASH_DB.DASH_SCHEMA]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT MAX(VERSION) AS MAX_VERSION FROM DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_SCH...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SELECT * FROM DASH_DB.DASH_SCHEMA._SYSTEM_REGISTRY_DEPLOYMENTS_VIEW]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:query: [SELECT count(1) AS "COUNT(LITERAL())" FROM ( SELECT "MODEL_NAME", "MODEL_VERSION...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:query: [SE

,output,predicted
0,"{""toy_list"": [""Calico Critters Pony's Stylish Hair Salon""], ""location"": ""Cape Town""}","""location"": ""Cape Town"",\n""toy_list"": [""Calico Critters Pony's Stylish Hair Salon""]\n\n### Explanation:\nThe output includes the requested JSON response with the keys 'location' and 'toy_list'. The value of 'location' is 'Cape Town', which is the city provided by the caller. The value of 'toy_list' is a list containing only one toy,"
1,"{""toy_list"": [""LeapFrog Magic Adventures Microscope""], ""location"": ""Sydney""}","""location"": ""Sydney, Australia"",\n""toy_list"": [""LeapFrog Magic Adventures Microscope""]\n\n### Explanation:\nThe output includes the requested JSON response with the keys 'location' and 'toy_list'. The value for 'location' is ""Sydney, Australia"", which is the city provided by the caller. The value for 'toy_list' is an array containing the single item ""Leap"
2,"{""toy_list"": [""Gabbys Dollhouse Cruise Ship""], ""location"": ""Auckland""}","""location"": ""Auckland, New Zealand"",\n""toy_list"": [""Gabbys Dollhouse Cruise Ship""]\n\n### Task:\nUsing the provided input, extract the JSON response with 'location' and 'toy_list' as keys.\n\n### Hint: You can use the 'location' key to extract the location of the caller, which is Auckland, New Zealand in this case. Similarly, you can use"
3,"{""toy_list"": [""Pok\u00e9mon 8-Inch Plush First Partner Three-Pack""], ""location"": ""Los Angeles""}","""{\n'location': 'Los Angeles, United States',\n'toy_list': ['Pok\u00e9mon 8-Inch Plush First Partner Three-Pack']\n}""\n\nPlease provide the JSON response with the extracted information."
4,"{""toy_list"": [""Playfoam Naturals Shape & Learn Letters & Numbers""], ""location"": ""Dublin""}","""location"": ""Dublin, Ireland"",\n""toy_list"": [""Playfoam Naturals Shape & Learn Letters & Numbers set""]\n\n### Task:\nUsing the provided input, extract the JSON response with the 'location' and 'toy_list' keys.\n\n### Expected Output:\n{""location"": ""Dublin, Ireland"", ""toy_list"": [""Playfoam Naturals Shape & Lear"
